In [1]:
library("xgboost")
library("Matrix")
library('Ckmeans.1d.dp')
library('lightgbm')



Attaching package: ‘lightgbm’


The following object is masked from ‘package:xgboost’:

    slice




## Read data and process data labels

In [2]:
time_matrix <- matrix(0,ncol = 3, nrow =4)
colnames(time_matrix) <- c("user_time", "system_time", "elapsed_time")
start_time = Sys.time()

In [3]:
data=read.csv('Weekly_features_matrix.csv')
data=data[,2:(dim(data)[2]-6)]

In [4]:
dim(data)

[1] 359  17

In [5]:
head(data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,length
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,1,0,1,0.9939369,6.320946e-11,36.725978,23.3448845,0.9600554,6.8563381,0.12476627,0.9982675,9.788630,-0.004831614,0.06449135,-0.45466261,0.2879798,2179
2,1,0,1,0.9946734,6.529056e-11,34.006351,19.8560557,0.9377499,5.8511863,0.09129043,0.9982275,9.779815,0.063312228,0.16413331,-0.36322842,0.3123634,1710
3,1,0,1,0.9991252,8.840224e-13,44.955510,11.0485829,0.6469218,2.6893773,0.02284238,0.9980612,9.806125,0.051096218,2.84368811,-0.07717025,2.4288344,2178
4,1,0,1,0.4159002,3.556671e-07,5.567640,-13.7061999,0.8181018,2.1703527,0.72989048,0.8936552,4.320394,-0.176902780,0.09272949,-0.55370413,0.3546872,2597
5,1,0,1,0.2103179,9.307235e-06,-12.975059,-1.0188984,0.6064015,0.7164726,0.87229229,0.6882600,1.428560,-0.261675675,0.10934385,-0.57273878,0.3857296,1603
6,1,0,1,0.4338204,1.481087e-06,6.188379,0.3520556,0.5692734,0.6017905,0.78743719,0.7555181,2.865383,-0.265161932,0.11337659,-0.56140364,0.3372221,1602


In [6]:
dlist= load('week_thetaf_datalist.RData')
datalist=eval(parse(text = dlist ))
res=datalist[[1]]
MASE=res[,,,6]
m=5

In [7]:
dim(MASE)

[1] 359   5   4

In [8]:
whichmin<-function(x){
    minx=min(x[x>0])
    loc=which(x==minx)[1]-1
    loc
}

meanunique=function(x)
    {
    mean(unique(x))
}

In [9]:
nanum=c()
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    a=apply(count,1,min)
    if(sum(is.na(a))>0)
        {
        nanum=append(nanum,i)
    }
    }

In [10]:
nanum

NULL

In [11]:
realbestmin=matrix(0,dim(MASE)[1],1)
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    min_value=apply(count,1,min)
    if (max(min_value,na.rm = TRUE)==0){
        realbestmin[i,]=0
        }
    else
        {
        min_value[is.na(min_value)]=100 
        realbestmin[i,]= whichmin(min_value)
    }
    }

In [12]:
table(realbestmin)

realbestmin
  0   1   2   3   4 
 72  31  68  47 141 

In [13]:
realbestmean=matrix(0,dim(MASE)[1],1)
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    mean_value=apply(count,1,meanunique)
    if (max(mean_value,na.rm = TRUE)==0){
        realbestmean[i,]=0
        }
    else
        {
        mean_value[is.na(mean_value)]=100 
        realbestmean[i,]= whichmin(mean_value)
    }
    }

In [14]:
realbestmean

4
4
4
1
0
0
0
0
0
2
0


In [15]:
head(data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,length
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,1,0,1,0.9939369,6.320946e-11,36.725978,23.3448845,0.9600554,6.8563381,0.12476627,0.9982675,9.788630,-0.004831614,0.06449135,-0.45466261,0.2879798,2179
2,1,0,1,0.9946734,6.529056e-11,34.006351,19.8560557,0.9377499,5.8511863,0.09129043,0.9982275,9.779815,0.063312228,0.16413331,-0.36322842,0.3123634,1710
3,1,0,1,0.9991252,8.840224e-13,44.955510,11.0485829,0.6469218,2.6893773,0.02284238,0.9980612,9.806125,0.051096218,2.84368811,-0.07717025,2.4288344,2178
4,1,0,1,0.4159002,3.556671e-07,5.567640,-13.7061999,0.8181018,2.1703527,0.72989048,0.8936552,4.320394,-0.176902780,0.09272949,-0.55370413,0.3546872,2597
5,1,0,1,0.2103179,9.307235e-06,-12.975059,-1.0188984,0.6064015,0.7164726,0.87229229,0.6882600,1.428560,-0.261675675,0.10934385,-0.57273878,0.3857296,1603
6,1,0,1,0.4338204,1.481087e-06,6.188379,0.3520556,0.5692734,0.6017905,0.78743719,0.7555181,2.865383,-0.265161932,0.11337659,-0.56140364,0.3372221,1602


In [16]:
set.seed(100)
index = sample(2,nrow(data),replace = TRUE,prob=c(0.7,0.3))

In [17]:
train_data=data[index==1,]
test_data=data[index==2,]
train_label_min=realbestmin[index==1,]
test_label_min=realbestmin[index==2,]
train_label_mean=realbestmean[index==1,]
test_label_mean=realbestmean[index==2,]

In [18]:
head(train_data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,length
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,1,0,1,0.9939369,6.320946e-11,36.725978,23.3448845,0.9600554,6.8563381,0.12476627,0.9982675,9.788630,-0.004831614,0.06449135,-0.45466261,0.2879798,2179
2,1,0,1,0.9946734,6.529056e-11,34.006351,19.8560557,0.9377499,5.8511863,0.09129043,0.9982275,9.779815,0.063312228,0.16413331,-0.36322842,0.3123634,1710
3,1,0,1,0.9991252,8.840224e-13,44.955510,11.0485829,0.6469218,2.6893773,0.02284238,0.9980612,9.806125,0.051096218,2.84368811,-0.07717025,2.4288344,2178
4,1,0,1,0.4159002,3.556671e-07,5.567640,-13.7061999,0.8181018,2.1703527,0.72989048,0.8936552,4.320394,-0.176902780,0.09272949,-0.55370413,0.3546872,2597
5,1,0,1,0.2103179,9.307235e-06,-12.975059,-1.0188984,0.6064015,0.7164726,0.87229229,0.6882600,1.428560,-0.261675675,0.10934385,-0.57273878,0.3857296,1603
6,1,0,1,0.4338204,1.481087e-06,6.188379,0.3520556,0.5692734,0.6017905,0.78743719,0.7555181,2.865383,-0.265161932,0.11337659,-0.56140364,0.3372221,1602


In [19]:
end_time = Sys.time()

In [20]:
time_matrix[1,]=end_time-start_time

## Target the interval where the actual error is minimum

In [21]:
start_time = Sys.time()

In [22]:
dtrain_xg_min_reg <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(train_label_min)) 
dtrain_xg_min_cl <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_min)) )

In [23]:
dtrain_lg_min_reg <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(train_label_min))
dtrain_lg_min_cl <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_min)))

In [24]:
xgb_min_reg <- xgboost(data = dtrain_xg_min_reg, nround=100)

[1]	train-rmse:1.870210 
[2]	train-rmse:1.421077 
[3]	train-rmse:1.111847 
[4]	train-rmse:0.864978 
[5]	train-rmse:0.697818 
[6]	train-rmse:0.594184 
[7]	train-rmse:0.506540 
[8]	train-rmse:0.447631 
[9]	train-rmse:0.393084 
[10]	train-rmse:0.356074 
[11]	train-rmse:0.334652 
[12]	train-rmse:0.318989 
[13]	train-rmse:0.303604 
[14]	train-rmse:0.273979 
[15]	train-rmse:0.257796 
[16]	train-rmse:0.237460 
[17]	train-rmse:0.221209 
[18]	train-rmse:0.204020 
[19]	train-rmse:0.192492 
[20]	train-rmse:0.180614 
[21]	train-rmse:0.169074 
[22]	train-rmse:0.160425 
[23]	train-rmse:0.146577 
[24]	train-rmse:0.138230 
[25]	train-rmse:0.131312 
[26]	train-rmse:0.120587 
[27]	train-rmse:0.114009 
[28]	train-rmse:0.103213 
[29]	train-rmse:0.090572 
[30]	train-rmse:0.082331 
[31]	train-rmse:0.079953 
[32]	train-rmse:0.075151 
[33]	train-rmse:0.070217 
[34]	train-rmse:0.065509 
[35]	train-rmse:0.057155 
[36]	train-rmse:0.055320 
[37]	train-rmse:0.049967 
[38]	train-rmse:0.046013 
[39]	train-rmse:0.045

In [25]:
xgb_min_cl <- xgboost(data = dtrain_xg_min_cl, nround=100, objective='multi:softmax',num_class=5)

[1]	train-mlogloss:1.250985 
[2]	train-mlogloss:0.998199 
[3]	train-mlogloss:0.826165 
[4]	train-mlogloss:0.695007 
[5]	train-mlogloss:0.595422 
[6]	train-mlogloss:0.516142 
[7]	train-mlogloss:0.443871 
[8]	train-mlogloss:0.391983 
[9]	train-mlogloss:0.339840 
[10]	train-mlogloss:0.304051 
[11]	train-mlogloss:0.270898 
[12]	train-mlogloss:0.242921 
[13]	train-mlogloss:0.213943 
[14]	train-mlogloss:0.196033 
[15]	train-mlogloss:0.177918 
[16]	train-mlogloss:0.163221 
[17]	train-mlogloss:0.150054 
[18]	train-mlogloss:0.139514 
[19]	train-mlogloss:0.128265 
[20]	train-mlogloss:0.119121 
[21]	train-mlogloss:0.109613 
[22]	train-mlogloss:0.102264 
[23]	train-mlogloss:0.096601 
[24]	train-mlogloss:0.090713 
[25]	train-mlogloss:0.085677 
[26]	train-mlogloss:0.080942 
[27]	train-mlogloss:0.077202 
[28]	train-mlogloss:0.073513 
[29]	train-mlogloss:0.070041 
[30]	train-mlogloss:0.067050 
[31]	train-mlogloss:0.064235 
[32]	train-mlogloss:0.061628 
[33]	train-mlogloss:0.059275 
[34]	train-mlogloss

In [26]:
lgb_min_reg <- lgb.train(data = dtrain_lg_min_reg, nrounds = 100)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004387 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1387
[LightGBM] [Info] Number of data points in the train set: 251, number of used features: 17
[LightGBM] [Info] Start training from score 2.438247
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

In [27]:
params <- list(objective = "multiclass",
               num_class = 5, 
               metric = "multi_logloss")
lgb_min_cl <- lgb.train(data = dtrain_lg_min_cl,nrounds = 100,params=params)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008428 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1387
[LightGBM] [Info] Number of data points in the train set: 251, number of used features: 17
[LightGBM] [Info] Start training from score -1.593627
[LightGBM] [Info] Start training from score -2.480931
[LightGBM] [Info] Start training from score -1.696812
[LightGBM] [Info] Start training from score -2.028945
[LightGBM] [Info] Start training from score -0.920283
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Lig

In [28]:
end_time = Sys.time()
time_matrix[2,]=end_time-start_time

## Target the interval where the average error is minimum

In [21]:
start_time = Sys.time()

In [22]:
dtrain_xg_mean_reg <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(train_label_mean)) 
dtrain_xg_mean_cl <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_mean)) )

In [23]:
dtrain_lg_mean_reg <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(train_label_mean))
dtrain_lg_mean_cl <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_mean)))

In [24]:
xgb_mean_reg <- xgboost(data = dtrain_xg_mean_reg, nround=100)

[1]	train-rmse:1.690535 
[2]	train-rmse:1.293743 
[3]	train-rmse:1.009612 
[4]	train-rmse:0.796056 
[5]	train-rmse:0.637363 
[6]	train-rmse:0.531038 
[7]	train-rmse:0.468428 
[8]	train-rmse:0.396996 
[9]	train-rmse:0.358474 
[10]	train-rmse:0.318009 
[11]	train-rmse:0.274160 
[12]	train-rmse:0.251820 
[13]	train-rmse:0.230670 
[14]	train-rmse:0.208318 
[15]	train-rmse:0.200065 
[16]	train-rmse:0.181147 
[17]	train-rmse:0.160560 
[18]	train-rmse:0.149841 
[19]	train-rmse:0.134564 
[20]	train-rmse:0.120426 
[21]	train-rmse:0.110400 
[22]	train-rmse:0.101426 
[23]	train-rmse:0.090734 
[24]	train-rmse:0.080885 
[25]	train-rmse:0.073019 
[26]	train-rmse:0.069362 
[27]	train-rmse:0.065416 
[28]	train-rmse:0.061734 
[29]	train-rmse:0.058379 
[30]	train-rmse:0.053623 
[31]	train-rmse:0.049947 
[32]	train-rmse:0.043781 
[33]	train-rmse:0.039962 
[34]	train-rmse:0.035805 
[35]	train-rmse:0.031591 
[36]	train-rmse:0.029531 
[37]	train-rmse:0.026145 
[38]	train-rmse:0.023604 
[39]	train-rmse:0.023

In [25]:
xgb_mean_cl <- xgboost(data = dtrain_xg_mean_cl, nround=100, objective='multi:softmax',num_class=5)

[1]	train-mlogloss:1.259651 
[2]	train-mlogloss:1.032255 
[3]	train-mlogloss:0.856995 
[4]	train-mlogloss:0.721542 
[5]	train-mlogloss:0.619608 
[6]	train-mlogloss:0.524581 
[7]	train-mlogloss:0.453645 
[8]	train-mlogloss:0.395014 
[9]	train-mlogloss:0.348195 
[10]	train-mlogloss:0.307030 
[11]	train-mlogloss:0.274133 
[12]	train-mlogloss:0.243495 
[13]	train-mlogloss:0.219642 
[14]	train-mlogloss:0.198236 
[15]	train-mlogloss:0.181067 
[16]	train-mlogloss:0.163478 
[17]	train-mlogloss:0.150263 
[18]	train-mlogloss:0.137407 
[19]	train-mlogloss:0.124838 
[20]	train-mlogloss:0.114863 
[21]	train-mlogloss:0.106150 
[22]	train-mlogloss:0.098601 
[23]	train-mlogloss:0.092236 
[24]	train-mlogloss:0.086517 
[25]	train-mlogloss:0.081625 
[26]	train-mlogloss:0.077204 
[27]	train-mlogloss:0.073708 
[28]	train-mlogloss:0.070326 
[29]	train-mlogloss:0.066951 
[30]	train-mlogloss:0.064508 
[31]	train-mlogloss:0.061921 
[32]	train-mlogloss:0.059710 
[33]	train-mlogloss:0.057734 
[34]	train-mlogloss

In [26]:
lgb_mean_reg <- lgb.train(data = dtrain_lg_mean_reg,nrounds = 100)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.040034 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1132
[LightGBM] [Info] Number of data points in the train set: 251, number of used features: 14
[LightGBM] [Info] Start training from score 2.103586
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

In [27]:
params <- list(objective = "multiclass",
               num_class = 5, 
               metric = "multi_logloss")
lgb_mean_cl <- lgb.train(data = dtrain_lg_mean_cl,params=params,nrounds = 100)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.315879 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1132
[LightGBM] [Info] Number of data points in the train set: 251, number of used features: 14
[LightGBM] [Info] Start training from score -1.291346
[LightGBM] [Info] Start training from score -2.480931
[LightGBM] [Info] Start training from score -1.536469
[LightGBM] [Info] Start training from score -2.158157
[LightGBM] [Info] Start training from score -1.168744
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

In [28]:
end_time = Sys.time()
time_matrix[3,]=end_time-start_time

## predict

In [37]:
start_time = Sys.time()

In [38]:
alldatalgb <- lgb.Dataset(data = as.matrix(data))
alldataxgb <- xgb.DMatrix(data = as.matrix(data))


In [39]:
xgbregmin=predict(xgb_min_reg,alldataxgb)
xgbclsmin=predict(xgb_min_cl,alldataxgb)
lgbregmin=predict(lgb_min_reg,as.matrix(data))
lgbclsmin=predict(lgb_min_cl,as.matrix(data))

In [40]:
xgbregmin

[1]  4.000281e+00  4.000470e+00  3.999863e+00  1.000291e+00  9.864192e-04
  [6]  1.998972e+00  1.729418e+00  5.658725e-05 -8.462324e-04  2.000752e+00
 [11]  2.000783e+00  1.292479e+00  3.999979e+00  3.999717e+00  3.737239e+00
 [16]  4.000157e+00  3.001029e+00  2.999781e+00 -2.542170e-05  3.000606e+00
 [21] -7.323622e-05  1.243336e+00  2.998985e+00  1.580991e+00 -2.538268e-04
 [26] -1.101574e-04  2.275365e+00  8.179777e-01  5.759504e-04  1.862427e-04
 [31]  1.154834e-03  5.143305e-01  1.998677e+00  2.061152e+00 -4.017608e-04
 [36]  3.398326e+00  4.000163e+00  4.000419e+00  1.901596e+00  1.999715e+00
 [41]  1.000250e+00  1.256742e+00  1.215858e+00  1.902699e+00  3.998973e+00
 [46]  4.000555e+00  3.543806e+00  3.342312e+00  3.998614e+00  4.001180e+00
 [51]  3.998754e+00  3.998912e+00  1.000080e+00 -4.809913e-05  3.000418e+00
 [56]  4.000386e+00 -7.340829e-05  2.000377e+00  7.899797e-04  3.999823e+00
 [61]  3.999472e+00  2.999822e+00  2.878354e+00  4.000842e+00  4.000255e+00
 [66]  3.999933e+00  3.999456e+00  1.000377e+00  9.992683e-01  1.999826e+00
 [71]  1.207320e-03  2.998296e+00  3.000470e+00  3.953344e+00  3.999865e+00
 [76]  1.338929e-03  2.823580e+00  2.972335e+00  2.368237e+00  3.998844e+00
 [81]  3.999104e+00  4.000328e+00  2.236562e+00  2.081876e+00  4.000533e+00
 [86]  3.998107e+00  1.926888e+00  3.999901e+00  1.000010e+00  2.613553e+00
 [91]  3.665034e+00  3.998754e+00  3.998624e+00 -1.028710e-03  1.233915e-01
 [96]  4.000016e+00  9.997854e-01  1.000610e+00  4.000645e+00  3.190357e+00
[101]  4.000204e+00  3.000252e+00  3.999805e+00  4.000062e+00  3.999392e+00
[106]  7.031659e-04  3.839874e+00  3.679376e+00  3.999335e+00  2.002084e+00
[111]  1.926888e+00  3.471982e+00  4.634083e-04  1.999735e+00  2.878584e+00
[116]  3.999610e+00  1.354034e+00  3.425640e+00  3.144946e-04  2.856190e-01
[121]  3.999862e+00  1.999125e+00  1.999700e+00  3.000384e+00  3.000284e+00
[126]  3.999878e+00  3.391855e+00  3.955925e+00  3.999672e+00  4.000337e+00
[131] -2.630988e-04  3.419558e+00  3.915268e+00  4.000076e+00  1.999898e+00
[136]  3.999123e+00  3.999592e+00  9.993287e-01  3.000730e+00  3.000095e+00
[141]  4.000448e+00  1.999030e+00  2.000709e+00  3.773542e+00  2.999748e+00
[146]  3.767282e+00  1.053928e+00  2.001159e+00  2.999811e+00  3.999205e+00
[151]  3.999799e+00  3.554614e+00  3.998224e+00  2.000353e+00  1.999699e+00
[156]  3.932625e+00  1.519352e+00  3.633693e+00  4.000247e+00  3.999566e+00
[161]  3.088506e+00  3.999365e+00  3.999652e+00  2.001617e+00 -1.337333e-04
[166]  2.330845e-04  1.999994e+00  3.561923e+00  3.063087e+00  2.954439e+00
[171]  3.999932e+00  3.998407e+00  4.000027e+00  3.368003e+00  3.439276e+00
[176]  3.999671e+00  3.153157e+00  3.999917e+00  2.000088e+00  3.998677e+00
[181]  2.001608e+00  2.041334e+00  2.419569e+00  4.000405e+00  3.999368e+00
[186] -6.867234e-05  3.998617e+00  1.000926e+00  4.000310e+00  1.357344e+00
[191]  2.882274e+00  2.000279e+00  1.000505e+00  4.793626e-01  3.999602e+00
[196]  3.000177e+00  2.001287e+00  3.002546e+00  2.000014e+00  1.533685e+00
[201]  3.998665e+00  1.748637e+00  2.975062e+00  2.334123e+00  3.999696e+00
[206]  9.401657e-01  2.335532e+00  3.999863e+00  3.000672e+00  2.001073e+00
[211]  4.000155e+00  3.096401e+00  1.999105e+00  1.999733e+00  3.999892e+00
[216]  4.000394e+00  1.266346e+00  2.916402e+00  3.997634e+00  2.999955e+00
[221]  3.999815e+00  3.999977e+00  3.999501e+00  2.966550e+00  1.461409e-03
[226]  2.999712e+00  3.055032e-04 -2.233701e-04  3.000226e+00  2.999969e+00
[231] -3.902987e-05  1.568190e-04  1.533578e+00 -2.551919e-04  7.037525e-04
[236]  1.575892e-03 -1.884032e-04  8.602477e-01  2.817605e+00  1.593828e+00
[241]  1.999637e+00  1.775297e+00 -1.961263e-04  2.428179e+00  4.231672e-04
[246]  1.540068e+00  1.789166e-03  3.999988e+00  3.999664e+00  3.999368e+00
[251]  3.999624e+00  3.701632e+00  3.999795e+00  4.000587e+00  2.142473e+00
[256]  3.509196e+00  3.860820e-05  9.998122e-01  2.000257e+00  9.735869e-01
[261]  2.001183e+00  3.705622e+00  1.014032e-03  2

In [41]:
xgbclsmin

[1] 4 4 4 1 0 2 0 0 0 2 2 4 4 4 4 4 3 3 0 3 0 0 3 0 0 0 0 0 0 0 0 0 2 4 0 4 4
 [38] 4 2 2 1 1 1 2 4 4 4 3 4 4 4 4 1 0 3 4 0 2 0 4 4 3 4 4 4 4 4 1 1 2 0 3 3 4
 [75] 4 0 2 2 4 4 4 4 2 2 4 4 3 4 1 2 4 4 4 0 0 4 1 1 4 3 4 3 4 4 4 0 4 4 4 2 3
[112] 4 0 2 4 4 2 4 0 0 4 2 2 3 3 4 4 4 4 4 0 4 4 4 2 4 4 1 3 3 4 2 2 4 3 4 2 2
[149] 3 4 4 4 4 2 2 4 2 4 4 4 2 4 4 2 0 0 2 4 2 2 4 4 4 4 4 4 4 4 2 4 2 4 2 4 4
[186] 0 4 1 4 2 4 2 1 2 4 3 2 3 2 1 4 4 4 4 4 0 4 4 3 2 4 2 2 2 4 4 1 4 4 3 4 4
[223] 4 3 0 3 0 0 3 3 0 0 0 0 0 0 0 0 4 0 2 0 0 3 0 0 0 4 4 4 4 4 4 4 1 4 0 1 2
[260] 4 2 4 0 0 0 0 0 3 0 2 2 4 4 0 2 3 4 3 2 2 4 0 2 0 4 3 2 4 4 4 4 4 1 4 0 2
[297] 3 0 2 2 4 4 4 2 3 3 3 2 0 4 2 1 0 3 3 0 0 0 4 4 3 4 1 3 2 3 4 1 4 4 4 1 3
[334] 4 4 3 1 1 2 0 1 1 4 2 4 2 2 4 0 0 0 4 1 4 0 2 4 1 4

In [42]:
lgbregmin

[1]  3.974765724  3.915294424  3.840040396  0.450179809  0.174063510
  [6]  1.652751412  1.933974490  0.073452289  0.323681318  2.307811631
 [11]  1.755703163  2.205533988  3.876431000  4.177420651  4.568216204
 [16]  4.394889667  3.389825383  3.117594830  0.374191540  2.980564492
 [21]  0.106977556  2.729061718  2.737708219  1.962748418 -0.003140835
 [26] -0.091133493  1.848156153  1.313585349  0.025281997  0.139794192
 [31]  0.489539368  0.776277514  1.462269745  3.114720824  0.078888240
 [36]  3.077456092  3.734518583  3.949057502  2.440253525  2.179193757
 [41]  1.053032971  1.853119340  1.631170098  2.407580847  3.610417560
 [46]  3.913574893  3.831274496  3.520691395  3.986934934  3.986934934
 [51]  3.912934449  3.607658103  1.849533786  0.599606104  3.109895949
 [56]  2.996372960  1.225520691  2.425361926  0.464757909  3.472607641
 [61]  4.017935934  2.679082625  3.407156518  4.106587424  4.233585670
 [66]  3.737606032  3.908664875  1.277488697  0.993004519  1.985563790
 [71]  1.203800495  2.694380855  2.829981893  5.046497109  3.958762182
 [76] -0.090758171  2.497636661  3.017029962  2.800060693  3.416934851
 [81]  3.708243572  3.957922072  2.416806803  2.527046854  3.591184282
 [86]  3.744974723  3.029692224  3.976234681  0.849932258  2.067243410
 [91]  3.682039277  4.129536992  3.208588306  0.101490859  1.136015562
 [96]  3.813044890  1.407862648  1.200295131  3.908079613  3.886672616
[101]  3.946873053  3.486849945  3.618062861  3.942103387  3.683884653
[106]  0.438475276  4.146005828  4.171114334  3.958886199  2.533167738
[111]  3.029692224  3.852143291  0.703578214  1.941276771  3.380954454
[116]  4.099274382  2.458744641  2.942548498  0.066534927  1.246311174
[121]  3.939423126  1.985694444  1.715605643  3.396642443  3.354193084
[126]  3.986380894  3.887881024  3.878123866  3.726978119  3.805716066
[131]  0.479777220  4.181918709  4.009461895  3.846655144  2.052667618
[136]  3.996796523  3.593512296  0.683494991  2.679121403  2.874646737
[141]  3.895815609  2.123600352  2.096904929  4.068639362  2.643126373
[146]  4.175731734  1.335653780  1.941939795  2.931902038  3.825399144
[151]  3.893974537  3.614647139  3.382536790  2.004818406  1.843987038
[156]  3.606450436  1.960676621  3.898402160  3.969461511  3.948396637
[161]  3.307285606  3.981164716  3.485969849  2.758097771  0.904082550
[166]  0.372863238  2.149003704  3.660547425  3.554765549  3.706849320
[171]  3.677213920  3.672330103  3.064017381  4.612856569  3.630410964
[176]  3.905955483  3.124089386  3.721517451  2.583184967  3.802727970
[181]  2.613733946  2.363243542  3.450551707  3.919510762  3.679337460
[186]  0.271798503  3.159261965  1.306155540  4.128846405  2.354485993
[191]  3.388819406  1.799271676  1.537920696  2.403915795  3.692724340
[196]  2.889708932  2.417562348  3.146837630  1.620897547  2.752485353
[201]  3.705958868  3.092472998  3.268983938  3.282330059  4.066758770
[206]  1.990254498  3.675647880  3.595067394  3.066118238  2.301510196
[211]  3.978285994  2.751515706  1.808217941  2.017963328  3.789329328
[216]  3.938182703  1.107250278  1.984498785  3.050176574  2.724320079
[221]  3.833943500  3.671571738  3.675782130  3.222394056  0.175035896
[226]  3.060862214  0.100581081  1.018308786  2.608767094  2.721419359
[231]  0.130250206  0.999681113  1.736712061  0.191576780  0.210560399
[236]  0.339252358  0.338961120  1.809218825  2.248080702  2.199605358
[241]  1.218645842  1.665741845 -0.146236603  2.441933110  0.447895059
[246]  2.450627463  0.300579973  4.169772369  4.206379308  4.009555689
[251]  3.054089651  4.003883397  4.107270427  4.013326776  1.374447317
[256]  3.239089364  0.624823830  0.673999828  2.013260571  3.233684522
[261]  2.239623404  3.513992185  0.171017700  1.250601084  1.445794929
[266]  1.157754889  0.212539631  2.908530926  0.778046831  3.253871853
[271]  2.752377834  3.655075931  3.717986438  0.651336194  1.722022725
[276]  2.537275795  3.578742159  2.605085217  2.342649489  2.571278113
[281]  3.082742870  0.4721

In [43]:
lgbclsmin

2.497017e-05,2.363057e-05,0.0014054164,7.874260e-03,9.906717e-01
5.034477e-05,1.232299e-04,0.0010319760,8.528831e-04,9.979416e-01
3.997912e-04,4.402409e-04,0.0066857566,9.523677e-04,9.915218e-01
1.945038e-02,9.785711e-01,0.0018455074,2.767828e-05,1.053489e-04
9.977718e-01,1.349066e-04,0.0009991067,3.403739e-04,7.538364e-04
3.470896e-02,2.773879e-04,0.9544429637,4.108836e-03,6.461848e-03
5.201361e-01,1.523790e-01,0.0360008056,1.825025e-01,1.089816e-01
9.982945e-01,2.778963e-04,0.0010611803,1.663793e-04,2.000920e-04
9.949102e-01,2.162053e-03,0.0019866455,2.327642e-04,7.083261e-04
1.021900e-04,3.147144e-04,0.9689338780,1.878449e-02,1.186473e-02
1.679571e-02,3.480908e-03,0.9622143110,3.591800e-04,1.714989e-02


In [44]:
datalength=dim(data)[1]
lgbclsm=matrix(lgbclsmin,m,datalength)
lgbclsminr=matrix(0,datalength,1)
for(i in 1:datalength)
    {
    lgbclsminr[i,]=which.max(lgbclsm[,i])
}
lgbclsminr=lgbclsminr-1

In [45]:
lgbclsminr

4
2
1
3
0
3
0
0
0
2
3


In [46]:
xgbregmean=predict(xgb_mean_reg,alldataxgb)
xgbclsmean=predict(xgb_mean_cl,alldataxgb)
lgbregmean=predict(lgb_mean_reg,as.matrix(data))
lgbclsmean=predict(lgb_mean_cl,as.matrix(data))

In [47]:
lgbclsm=matrix(lgbclsmean,m,datalength)
lgbclsmeanr=matrix(0,datalength,1)
for(i in 1:datalength)
    {
    lgbclsmeanr[i,]=which.max(lgbclsm[,i])
}
lgbclsmeanr=lgbclsmeanr-1

In [48]:
preallmin=cbind(xgbclsmin,xgbregmin)
preallmin=cbind(preallmin,lgbclsminr)
preallmin=cbind(preallmin,lgbregmin)

In [49]:
colnames(preallmin)=c('xgbclsmin','xgbregmin','lgbclsmin','lgbregmin')
head(preallmin)

xgbclsmin,xgbregmin,lgbclsmin,lgbregmin
4,4.0002808571,4,3.9747657
4,4.0004696846,2,3.9152944
4,3.9998633862,1,3.8400404
1,1.0002913475,3,0.4501798
0,0.0009864192,0,0.1740635
2,1.9989718199,3,1.6527514


In [50]:
preallmean=cbind(xgbclsmean,xgbregmean)
preallmean=cbind(preallmean,lgbclsmeanr)
preallmean=cbind(preallmean,lgbregmean)

In [51]:
head(preallmean)

xgbclsmean,xgbregmean,,lgbregmean
4,3.9997739792,4,3.87829283
4,3.9997663498,0,3.83083941
4,3.9998736382,0,3.88001566
1,0.9997478724,3,0.61846058
0,-0.0002057868,4,0.02797209
0,0.0002924836,0,0.07197927


In [52]:
colnames(preallmean)=c('xgbclsmean','xgbregmean','lgbclsmean','lgbregmean')
head(preallmean)

xgbclsmean,xgbregmean,lgbclsmean,lgbregmean
4,3.9997739792,4,3.87829283
4,3.9997663498,0,3.83083941
4,3.9998736382,0,3.88001566
1,0.9997478724,3,0.61846058
0,-0.0002057868,4,0.02797209
0,0.0002924836,0,0.07197927


In [53]:
end_time = Sys.time()
time_matrix[4,]=end_time-start_time

In [54]:
result_list=list(preallmin,preallmean,time_matrix)

In [59]:
save(result_list, file = "weekly_thetaf_opt_pre_result.RData")

In [60]:
time_matrix

user_time,system_time,elapsed_time
3.7210128,3.7210128,3.7210128
2.7225070,2.7225070,2.7225070
2.4123126,2.4123126,2.4123126
0.8011491,0.8011491,0.8011491


In [29]:
importance_matrix1 <- xgb.importance(model = xgb_mean_cl)
importance_matrix2 <- xgb.importance(model = xgb_mean_reg)
importance_matrix3 <- lgb.importance(lgb_mean_cl)
importance_matrix4 <- lgb.importance(lgb_mean_reg)

importance_matrix1=importance_matrix1[order(importance_matrix1$Feature),]
importance_matrix2=importance_matrix2[order(importance_matrix2$Feature),]
importance_matrix3=importance_matrix3[order(importance_matrix3$Feature),]
importance_matrix4=importance_matrix4[order(importance_matrix4$Feature),]

importance_matrix_mean=(importance_matrix1[,2:4]+importance_matrix2[,2:4]+importance_matrix3[,2:4]+importance_matrix4[,2:4])/4

importance=cbind(importance_matrix1[,1],importance_matrix_mean)

write.csv(importance,'w_thetaf_imp_mean.csv')